<a href="https://colab.research.google.com/github/SisseBJ/Caldiss-SDS-phd-19/blob/master/Portfolio_2_Thursday_Real_Trump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Realtrump

In [0]:
import pandas as pd
import numpy as np

In [0]:
data_df = pd.read_csv("https://raw.githubusercontent.com/DeepLearnI/trump_tweet_classifier/master/code/tweet_labels.csv", sep=",")
data_df.head()

,tweet,labels
0,To every one of the HEROES we recognized today...,True
1,We have been serving as policemen in Afghanist...,True
2,"...the Economy, where there is NO Recession, m...",True
3,"...the look of turmoil in the White House, of ...",True
4,A lot of Fake News is being reported that I ov...,True


In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
!pip install -qq -U gensim

     |████████████████████████████████| 24.2MB 88kB/s 


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
# Import the dictionary builder
from gensim.corpora.dictionary import Dictionary

In [0]:
# Import the TfidfModel from Gensim
from gensim.models.tfidfmodel import TfidfModel

In [0]:
# Just like before, we import the model
from gensim.models.lsimodel import LsiModel

In [0]:
# Import stopwords
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [0]:
# Split the data using the train_test_split module. We keep 20% of the data for testing and use 80% to train the model
# Random state defined with an arbitrary number for reproducibility
train_df, test_df = train_test_split(data_df, test_size = 0.2, random_state = 42)

In [0]:
# Tokenize our texts and remove stopwords, also kick out numbers, lower everything

 train_tokens = train_df['tweet'].map(lambda t: [tok.lower() for tok in word_tokenize(t) if tok not in stop_words and tok.isalpha()])

In [0]:
test_tokens = test_df['tweet'].map(lambda t: [tok.lower() for tok in word_tokenize(t) if tok not in stop_words and tok.isalpha()])

In [0]:
# Generate a dictionary
dictionary = Dictionary(train_tokens)

In [0]:
# Filter it for extreme stuff
dictionary.filter_extremes(no_below = 10, no_above=0.4)

In [0]:
# construct corpus using this dictionary
train_corpus = [dictionary.doc2bow(doc) for doc in train_tokens]
test_corpus = [dictionary.doc2bow(doc) for doc in test_tokens]

In [0]:
# Tooling to map between corpus (gensim) and matrix - more general
from gensim.matutils import corpus2csc, corpus2dense

In [0]:
X_train = corpus2csc(train_corpus)
X_test = corpus2csc(test_corpus)

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
model = LogisticRegression()

In [0]:
model.fit(X_train.T, train_df.labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
model.score(X_test.T, test_df.labels)

0.7055374592833876